In [1]:
#import dependencies
import requests
import json
from config import api_key
from pprint import pprint
import pandas as pd
from time import sleep
from tqdm import tqdm

In [2]:
#import csv file and convert to dataframe
movie_csv = "../../Extract/Movies/MoviesOnStreamingPlatforms_updated.csv" 
movies = pd.read_csv(movie_csv)

In [3]:
movies.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [4]:
#check to see count of dataframe
movies.count()

Unnamed: 0         16744
ID                 16744
Title              16744
Year               16744
Age                 7354
IMDb               16173
Rotten Tomatoes     5158
Netflix            16744
Hulu               16744
Prime Video        16744
Disney+            16744
Type               16744
Directors          16018
Genres             16469
Country            16309
Language           16145
Runtime            16152
dtype: int64

In [5]:
movies.dtypes

Unnamed: 0           int64
ID                   int64
Title               object
Year                 int64
Age                 object
IMDb               float64
Rotten Tomatoes     object
Netflix              int64
Hulu                 int64
Prime Video          int64
Disney+              int64
Type                 int64
Directors           object
Genres              object
Country             object
Language            object
Runtime            float64
dtype: object

In [6]:
#filter out movies on Prime Video
movies_filtered = movies[(movies['Prime Video'] == 0)]                  

In [7]:
#check the count to see how much is left over after filtering out Prime Video
movies_filtered.count()

Unnamed: 0         4390
ID                 4390
Title              4390
Year               4390
Age                2375
IMDb               4265
Rotten Tomatoes    2081
Netflix            4390
Hulu               4390
Prime Video        4390
Disney+            4390
Type               4390
Directors          4191
Genres             4297
Country            4231
Language           4208
Runtime            4158
dtype: int64

In [8]:
#filter columns to only get the Title and Year columns
movie_title = movies_filtered["Title"]
movie_year = movies_filtered["Year"]

In [9]:
#set the url for the IMDB api and use the api_key from the config.py
url = "http://www.omdbapi.com/?t="
apikey = "&apikey=" + api_key

In [10]:
#create empty list to store json objects into
movie_api_list = []

#for loop to go through each movie in Title column and store the json object into the movie_api_list list
for index, row in tqdm(movies_filtered.iterrows()):
    
    movie = row["Title"]
    year = row["Year"]
    
    url_movie = (url + movie + "&y=" + str(year) + apikey )
    
    response = requests.get(url_movie)
    data = response.json()
    df = pd.json_normalize(data)
    movie_api_list.append(df)
    

4390it [03:01, 24.24it/s]


In [11]:
#convert list of json objects into dataframe. Without ignore_index = True the index will be 0 for all rows
movies_df = pd.concat(movie_api_list, ignore_index=True)

In [12]:
#check to see the columns
movies_df.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Response', 'Error',
       'totalSeasons'],
      dtype='object')

In [13]:
#select only the columns we want
movies_df_clean = movies_df[["Title", "Year", "Director", "Genre", "imdbRating", "imdbVotes", "Ratings", "Country", "Awards" ]].copy()

In [14]:
#display 
movies_df_clean.head()

,Title,Year,Director,Genre,imdbRating,imdbVotes,Ratings,Country,Awards
0,Inception,2010,Christopher Nolan,"Action, Adventure, Sci-Fi, Thriller",8.8,"2,029,549","[{'Source': 'Internet Movie Database', 'Value'...","USA, UK",Won 4 Oscars. Another 152 wins & 218 nominations.
1,The Matrix,1999,"Lana Wachowski, Lilly Wachowski","Action, Sci-Fi",8.7,"1,647,373","[{'Source': 'Internet Movie Database', 'Value'...",USA,Won 4 Oscars. Another 37 wins & 51 nominations.
2,Avengers: Infinity War,2018,"Anthony Russo, Joe Russo","Action, Adventure, Sci-Fi",8.4,"812,007","[{'Source': 'Internet Movie Database', 'Value'...",USA,Nominated for 1 Oscar. Another 46 wins & 75 no...
3,Back to the Future,1985,Robert Zemeckis,"Adventure, Comedy, Sci-Fi",8.5,"1,039,498","[{'Source': 'Internet Movie Database', 'Value'...",USA,Won 1 Oscar. Another 19 wins & 25 nominations.
4,Spider-Man: Into the Spider-Verse,2018,"Bob Persichetti, Peter Ramsey, Rodney Rothman","Animation, Action, Adventure, Family, Sci-Fi",8.4,"344,012","[{'Source': 'Internet Movie Database', 'Value'...",USA,Won 1 Oscar. Another 79 wins & 55 nominations.


In [15]:
#split the Ratings column which has list of dictionaries
movies_df_clean = pd.concat([movies_df_clean.drop(['Ratings'], axis=1), movies_df_clean['Ratings'].apply(pd.Series)], axis=1)

In [16]:
#split 1 which is rotten tomatoes
movies_df_clean = pd.concat([movies_df_clean.drop([1], axis=1), movies_df_clean[1].apply(pd.Series)], axis=1)

In [17]:
#drop 0 and Source columns
movies_df_clean = movies_df_clean.drop(columns = [0, "Source"])

In [18]:
#rename value to RottenTomatoes
movies_df_clean = movies_df_clean.rename(columns={"Value": "RottenTomatoes"})

In [19]:
#split 2 which is MetaCritic
movies_df_clean = pd.concat([movies_df_clean.drop([2], axis=1), movies_df_clean[2].apply(pd.Series)], axis=1)

In [20]:
#drop 0 and Source
movies_df_clean = movies_df_clean.drop(columns = [0, "Source"])

In [21]:
#rename value to MetaCritic
movies_df_clean = movies_df_clean.rename(columns={"Value": "MetaCritic"})

In [22]:
#remove percentage sign from RottenTomatoes
movies_df_clean["RottenTomatoes"] = movies_df_clean['RottenTomatoes'].str.rstrip('%')

#remove /100 from RottenTomatoes because some of them are /100 instead of %
movies_df_clean["RottenTomatoes"] = movies_df_clean['RottenTomatoes'].str.replace('/100', '')

In [23]:
#remove /100 from MetaCritic
movies_df_clean["MetaCritic"] = movies_df_clean['MetaCritic'].str.replace('/100', '') 

In [24]:
#rename imdbRating and imdbVotes for consistency in the database
movies_df_clean = movies_df_clean.rename(columns={"imdbRating": "IMDB_Rating", "imdbVotes": "IMDB_Votes"})

In [25]:
#remove comma in IMDB_Votes
movies_df_clean["IMDB_Votes"] = movies_df_clean['IMDB_Votes'].str.replace(',', '')
#movies_df_clean["Year"] = movies_df_clean['Year'].str.replace('-', '')

In [26]:
#drop NaN rows
movies_df_clean = movies_df_clean.dropna(how = "all")

In [27]:
#replace all N/A with 0
movies_df_clean = movies_df_clean.replace("N/A", "0")

In [28]:
#fill NaN to 0 so it can be converted to int or float
movies_df_clean["IMDB_Votes"].fillna(0, inplace = True)
movies_df_clean["RottenTomatoes"].fillna(0, inplace = True)
movies_df_clean["MetaCritic"].fillna(0, inplace = True)

In [29]:
#strip out "-" from Year as some Movies have "-" at the end of the year
movies_df_clean["Year"] = movies_df_clean['Year'].str.rstrip('–')

In [30]:
#create the key by combining the Title and the Year
movies_df_clean['key'] = movies_df_clean['Title'] + movies_df_clean['Year']

In [31]:
#convert columns to float or int. need to convert to string first
movies_df_clean["IMDB_Votes"] = movies_df_clean['IMDB_Votes'].astype(str).astype(int)
movies_df_clean["IMDB_Rating"] = movies_df_clean["IMDB_Rating"].astype(str).astype(float)
movies_df_clean["RottenTomatoes"] = movies_df_clean["RottenTomatoes"].astype(str).astype(int)
movies_df_clean["MetaCritic"] = movies_df_clean["MetaCritic"].astype(str).astype(int)

In [32]:
#check data types
movies_df_clean.dtypes

Title              object
Year               object
Director           object
Genre              object
IMDB_Rating       float64
IMDB_Votes          int32
Country            object
Awards             object
RottenTomatoes      int32
MetaCritic          int32
key                object
dtype: object

In [33]:
#display
movies_df_clean.head()

,Title,Year,Director,Genre,IMDB_Rating,IMDB_Votes,Country,Awards,RottenTomatoes,MetaCritic,key
0,Inception,2010,Christopher Nolan,"Action, Adventure, Sci-Fi, Thriller",8.8,2029549,"USA, UK",Won 4 Oscars. Another 152 wins & 218 nominations.,87,74,Inception2010
1,The Matrix,1999,"Lana Wachowski, Lilly Wachowski","Action, Sci-Fi",8.7,1647373,USA,Won 4 Oscars. Another 37 wins & 51 nominations.,88,73,The Matrix1999
2,Avengers: Infinity War,2018,"Anthony Russo, Joe Russo","Action, Adventure, Sci-Fi",8.4,812007,USA,Nominated for 1 Oscar. Another 46 wins & 75 no...,85,68,Avengers: Infinity War2018
3,Back to the Future,1985,Robert Zemeckis,"Adventure, Comedy, Sci-Fi",8.5,1039498,USA,Won 1 Oscar. Another 19 wins & 25 nominations.,96,87,Back to the Future1985
4,Spider-Man: Into the Spider-Verse,2018,"Bob Persichetti, Peter Ramsey, Rodney Rothman","Animation, Action, Adventure, Family, Sci-Fi",8.4,344012,USA,Won 1 Oscar. Another 79 wins & 55 nominations.,97,87,Spider-Man: Into the Spider-Verse2018


In [34]:
#save the csv
movies_df_clean.to_csv("movies_df_clean.csv", encoding="utf-8")